# Intro & Loading 

In [2]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio.utils import helper as h
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display


%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

In [19]:
samplesetname = "21Q1"

virtual= {}
virtual['public'] = 'public-21q1-4b39'
virtual['ibm'] = 'ibm-21q1-abd9'
virtual['dmc'] = 'dmc-21q1-0e11'
virtual['internal']='internal-21q1-4fc4'


taiga_mutation = {}
taiga_mutation['internal'] = "depmap-mutation-calls-9be3"
taiga_mutation['ibm'] = "mutations-b05c"
taiga_mutation['dmc'] = "depmap-mutation-calls-dfce"
taiga_mutation['public'] ="depmap-mutation-calls-9a1a"

taiga_expression = {}
taiga_expression['internal'] ="depmap-rnaseq-expression-data-363a"
taiga_expression['ibm'] = "expression-9d97"
taiga_expression['dmc'] = "depmap-rnaseq-expression-data-80ef"
taiga_expression['public'] ="depmap-rnaseq-expression-data-ccd0"

taiga_fusion = {}
taiga_fusion['internal'] = "gene-fusions-8b7a"
taiga_fusion['ibm'] = "fusion-dc08"
taiga_fusion['dmc'] = "gene-fusions-375f"
taiga_fusion['public'] ="gene-fusions-6212"

taiga_copynumber = {}
taiga_copynumber['internal'] ="depmap-wes-cn-data-81a7"
taiga_copynumber['ibm'] = "cn-e20f"
taiga_copynumber['dmc'] = "depmap-cn-data-9b9d"
taiga_copynumber['public'] = "depmap-wes-cn-data-97cc"

prev_virtual = {}
#prev_virtual['public'] = 'public-20q3-3d35'
#prev_virtual['dmc'] = 'dmc-20q3-deprecated-never-released--5f55'
#prev_virtual['internal'] = 'internal-20q3-00d0'

prev_virtual['public'] = 'public-20q4-a4b3'
prev_virtual['dmc'] = 'dmc-20q4-fcf4'
#prev_virtual['ibm'] = 'ibm-20q4-269f'
prev_virtual['internal'] = 'internal-20q4-2540'
potential_list_url = "https://docs.google.com/spreadsheets/d/1YuKEgZ1pFKRYzydvncQt9Y_BKToPlHP-oDB-0CAv3gE"

release = samplesetname.lower()

In [20]:
new = {}
gsheets = sheets.get(potential_list_url).sheets[0].to_frame()
new['internal'] = set([i for i in gsheets['Internal'].values.tolist() if str(i) != "nan"])
new['dmc'] = set([i for i in gsheets['DMC'].values.tolist() if str(i) != "nan"])
new['ibm'] = set([i for i in gsheets['IBM'].values.tolist() if str(i) != "nan"])
new['public'] = set([i for i in gsheets['Public'].values.tolist() if str(i) != "nan"])

new["internal"] = new["internal"] | new["ibm"] | new["dmc"] | new["public"]
new["ibm"] = new["ibm"] | new["dmc"] | new["public"]
new["dmc"] = new["dmc"] | new["public"]

## Getting what was released before

In [21]:
prevmut = {}
prevrna = {}
prevcn = {}
prevwes = {}
prev = {}
for val in ['internal', 'dmc', 'public']:
    print(val)
    prevmut[val] = set(tc.get(name=prev_virtual[val], file='CCLE_mutations').DepMap_ID)
    prevrna[val] = set(tc.get(name=prev_virtual[val], file='CCLE_expression').index)
    prevcn[val] = set(tc.get(name=prev_virtual[val], file='CCLE_segment_cn').DepMap_ID)
    prev[val] = prevmut[val] | prevrna[val] | prevcn[val]
    prevwes[val] = prevmut[val] | prevcn[val]

internal
dmc
public


In [ ]:
print('in cn but not mut')
print(prevwes["internal"] - prevmut["internal"])
print('in mut but not cn')
print(prevwes["internal"] - prevcn["internal"])
print('in rna but no wes/wgs')
print(prev["internal"] - prevwes["internal"])
print('in wes/wgs but not rna')
print(prev["internal"] - prevrna["internal"])

In [23]:
prev['ibm'] = prev['dmc']
prevcn['ibm'] = prevcn['dmc']
prevmut['ibm'] = prevmut['dmc']
prevrna['ibm'] = prevrna['dmc']

## managing the readmes

In [6]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

fatal: destination path 'depmap-release-readmes' already exists and is not an empty directory.


In [134]:
! cd ../depmap-release-readmes && git pull

Updating bb67687..fa199c0
Fast-forward
 release-20q4/dmc-20q4.txt      | 41 +---------------------
 release-20q4/internal-20q4.txt | 77 ++++++++++++++++++++----------------------
 release-20q4/public-20q4.txt   | 74 ++++++++++++++++++++--------------------
 3 files changed, 74 insertions(+), 118 deletions(-)


In [143]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

Making public
Making internal
Making dmc
[master 87f7b1c] 21Q1
 4 files changed, 140 insertions(+), 454 deletions(-)
 delete mode 100644 release-21q1/ibm-21q1.txt
Counting objects: 6, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 2.63 KiB | 0 bytes/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/broadinstitute/depmap-release-readmes.git
   8cb35f2..87f7b1c  master -> master


In [ ]:
# NOW UPDATE THE READMEs

In [144]:
! mkdir temp/README/

mkdir: cannot create directory ‘temp/README/’: File exists


In [149]:
! cd ../depmap-release-readmes && cp release-$release/* ../ccle_processing/readmes/ && git add . && git commit -m "Omics: updating readmes to new release" && git push

On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working tree clean


# Mutations

## Somatic

In [12]:
mutations = pd.read_csv("temp/wes_somatic_mutations_withlegacy_"+samplesetname+".csv")
damaging = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv', index_col=0)
othercons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv', index_col=0)
othernoncons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv', index_col=0)
hotspot = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv', index_col=0)
hotspot=hotspot.astype(int)
damaging=damaging.astype(int)
othercons=othercons.astype(int)
othernoncons=othernoncons.astype(int)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,22,25,28,29,31,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#reverting to previous versions
mutations = mutations[mutations.is_likely_immortalization!=True]
mutations = mutations[['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Allele', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'DepMap_ID', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious',
       'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt',
       'ExAC_AF',"Variant_annotation", 'CGA_WES_AC', 'HC_AC',
       'RD_AC', 'RNAseq_AC', 'SangerWES_AC', 'WGS_AC']].rename(columns={"Tumor_Allele":"Tumor_Seq_Allele1"})

In [14]:
! ls temp/README

temp/README


In [11]:
blacklist = set()
for val in ['internal', 'ibm', "dmc", "public"]:
    print('_________________________________________________')
    print(val)
    missing = set(new[val]) - set(mutations.DepMap_ID)
    print('nott present')
    removed = set(prev[val]) - set(mutations.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevmut[val]) - set(mutations.DepMap_ID)
    print(removed)
    blacklist = (set(mutations.DepMap_ID) - (prev[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(blacklist)
    # adding files
    a = mutations[~mutations.DepMap_ID.isin(blacklist)]
    print(len(mutations)- a)
    a.to_csv('temp/all_somatic_mutations_withlegacy.csv', index=False)
    a = damaging[~damaging.DepMap_ID.isin(blacklist)]
    print(len(damaging.columns)-a)
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
    a = othercons[~othercons.DepMap_ID.isin(blacklist)]
    print(len(othercons.columns)-a)
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
    a = othernoncons[~othernoncons.DepMap_ID.isin(blacklist)]
    print(len(othernoncons.columns)-a)
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
    a = hotspot[~hotspot.DepMap_ID.isin(blacklist)]
    print(len(hotspot.columns)-a)
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    
    # updating on taiga
    tc.update_dataset(dataset_permaname=taiga_mutation[val],
                     upload_file_path_dict={
    'temp/all_somatic_mutations_withlegacy.csv': 'TableCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv': 'NumericMatrixCSV',
    'temp/README': 'Raw'},
                     dataset_description="""
    #Mutations Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLISTED:
    """+str(blacklist))

    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_mutation[val], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot'), ('README','README')])

_________________________________________________
internal
nott present
{'ACH-001429', 'ACH-002359', 'ACH-001087', 'ACH-001225', 'ACH-001712', 'ACH-002378', 'ACH-001088', 'ACH-001015', 'ACH-001079', 'ACH-001198', 'ACH-002341', 'ACH-002315', 'ACH-001171', 'ACH-001224', 'ACH-002014', 'ACH-001121', 'ACH-001108', 'ACH-001045', 'ACH-001743', 'ACH-001017', 'ACH-001037', 'ACH-001018', 'ACH-001187', 'ACH-001316', 'ACH-002709', 'ACH-001071', 'ACH-001000', 'ACH-001393', 'ACH-001175', 'ACH-001741', 'ACH-001189', 'ACH-001011', 'ACH-002303', 'ACH-001101', 'ACH-002335', 'ACH-001249', 'ACH-002512'}
removed
{'ACH-002359', 'ACH-001108', 'ACH-002341', 'ACH-001189', 'ACH-002315', 'ACH-001011', 'ACH-002303', 'ACH-001187', 'ACH-002378'}
missing
{'ACH-002084', 'ACH-001992', 'ACH-001675'}
blacklist
{'ACH-001227', 'ACH-002476', 'ACH-001760', 'ACH-002138', 'ACH-003210', 'ACH-001759', 'ACH-001686', 'ACH-001756', 'ACH-001758', 'ACH-001434'}
4982
0
0
0
0
Uploading all_somatic_mutations_withlegacy...
hitting https

KeyboardInterrupt: 

In [ ]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_mutation['internal'], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot')])

# Copy Number

In [26]:
genecn= pd.read_csv('temp/all_'+samplesetname+'_gene_cn.csv',index_col=0)
segmentcn = pd.read_csv('temp/all_'+samplesetname+'_segment.csv')

In [13]:
#genecn = genecn.apply(lambda x: (x**2)-1)

In [ ]:
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prevwes[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(segmentcn.DepMap_ID)
    blacklist = set(segmentcn.DepMap_ID) - (prevcn[val] | set(new[val]))
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## for segment removing first blacklisted, then embargoed, to create two datasets
    print(len(segmentcn))
    segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
    print(len(segmentcn))
    segmentcn.to_csv('temp/all_merged_segments.csv', index=False)
    print(len(genecn))
    genecn = genecn[~genecn.index.isin(blacklist)]
    print(len(genecn))
    genecn.to_csv('temp/all_merged_genes_cn.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # Add to Taiga
    tc.update_dataset(dataset_permaname=taiga_copynumber[val], 
                      upload_file_path_dict={
                        'temp/all_merged_genes_cn.csv': 'NumericMatrixCSV',
                        'temp/all_merged_segments.csv': 'TableCSV',
                        'temp/README': 'Raw'},
                      dataset_description=
    """
    #Copy Number Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLIST:
    """+str(blacklist))
    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_copynumber[val], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn','all_merged_segments')])

In [ ]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_copynumber['internal'], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn', 'all_merged_segments')])

# RNA

In [15]:
transcripts_tpm = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_tpm.csv',index_col=0)
genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_genes_tpm.csv',index_col=0)
genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_genes_expected_count.csv',index_col=0)
proteincoding_genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_expected_count.csv',index_col=0)
proteincoding_genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_tpm.csv',index_col=0)
transcripts_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_expected_count.csv',index_col=0)
#putting it back to what it was before

# log transforming tpm data
transcripts_tpm=transcripts_tpm.apply(lambda x: np.log2(x+1))
genes_tpm=genes_tpm.apply(lambda x: np.log2(x+1))
proteincoding_genes_tpm=proteincoding_genes_tpm.apply(lambda x: np.log2(x+1))

In [17]:
%store -r rnafailed

In [24]:
blacklist = set()
for val in ['internal','ibm', 'dmc','public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(genes_tpm.index)
    print(removed)
    #print('removed for QC reasons')
    #print(set(rename.keys()))
    print('removed')
    removed = set(prevrna[val]) - set(genes_tpm.index)
    #print(removed - set(rename.keys()))
    missing = set(new[val]) - set(genes_tpm.index)
    blacklist = (set(genes_tpm.index) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)

    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(genes_expected_count))
    a = genes_expected_count[~genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_expected_count.csv')
    print(len(genes_tpm))
    a = genes_tpm[~genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_tpm.csv')
    print(len(proteincoding_genes_tpm))
    a = proteincoding_genes_tpm[~proteincoding_genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_tpm.csv')
    print(len(transcripts_tpm))
    a = transcripts_tpm[~transcripts_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_tpm.csv')
    print(len(proteincoding_genes_expected_count))
    a = proteincoding_genes_expected_count[~proteincoding_genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_expected_count.csv')
    print(len(transcripts_expected_count))
    a = transcripts_expected_count[~transcripts_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_expected_count.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # adding to taiga
    tc.update_dataset(dataset_permaname=taiga_expression[val],
                     upload_file_path_dict={
                       'temp/expression_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_expected_count.csv': 'NumericMatrixCSV',
                     "temp/README": "Raw"},
                      dataset_description=
    """
    # INTERNAL RNA

    for information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str("rename")+"""

    BLACKLIST:
    """+str(blacklist))

    # add to virtual 
    AddToVirtual(virtual[val], taiga_expression[val], files=[
    ('CCLE_expression_full', 'expression_genes_tpm'), 
    ('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
    ('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
    ('CCLE_expression', 'expression_proteincoding_genes_tpm'),
    ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),
    ('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

_________________________________________________
internal
not present
{'ACH-001208', 'ACH-002233', 'ACH-001087', 'ACH-002179', 'ACH-002153', 'ACH-002125', 'ACH-001042', 'ACH-002117', 'ACH-001282', 'ACH-002248', 'ACH-002341', 'ACH-001234', 'ACH-002157', 'ACH-001092', 'ACH-002280', 'ACH-002380', 'ACH-002312', 'ACH-001037', 'ACH-002119', 'ACH-002091', 'ACH-002299', 'ACH-002363', 'ACH-001175', 'ACH-002020', 'ACH-002389', 'ACH-002185', 'ACH-002200', 'ACH-001639', 'ACH-002354', 'ACH-002165', 'ACH-002212', 'ACH-002191', 'ACH-002336', 'ACH-001088', 'ACH-002098', 'ACH-001024', 'ACH-001680', 'ACH-002400', 'ACH-001417', 'ACH-002347', 'ACH-002394', 'ACH-002105', 'ACH-002154', 'ACH-001064', 'ACH-002293', 'ACH-002261', 'ACH-002163', 'ACH-002097', 'ACH-002134', 'ACH-002290', 'ACH-002202', 'ACH-002395', 'ACH-002268', 'ACH-002159', 'ACH-002297', 'ACH-002132', 'ACH-002251', 'ACH-001131', 'ACH-002271', 'ACH-002103', 'ACH-001331', 'ACH-002230', 'ACH-002345', 'ACH-002401', 'ACH-001198', 'ACH-002282', 'ACH

	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to d

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

...

	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to d

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: expression_genes_tpm properly converted and uploaded
Up

	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to d

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

In [ ]:
AddToVirtual('depmap-a0ab', taiga_expression['internal'], files=[
('CCLE_expression_full', 'expression_genes_tpm'), 
('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
('CCLE_expression', 'expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

# Fusions

In [55]:
fusions=pd.read_csv('temp/unfiltered_fusions_'+samplesetname+'.csv')
filtered=pd.read_csv('temp/fusions_'+samplesetname+'.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'temp/unfiltered_fusions_20Q4.csv'

In [56]:
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(fusions.DepMap_ID)
    print(removed)
    print('removed for QC reasons')
    print(set(rename.keys()))
    print('removed')
    removed = set(prevrna[val]) - set(fusions.DepMap_ID)
    print(removed - set(rename.keys()))
    missing = set(new[val]) - set(fusions.DepMap_ID)
    blacklist = set(fusions.DepMap_ID) - (prev[val] | set(new[val]))
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(fusions))
    fusions = fusions[~fusions.DepMap_ID.isin(blacklist)]
    print(len(fusions))
    fusions.to_csv('temp/fusions.csv', index=False)
    print(len(filtered))
    filtered= filtered[~filtered.DepMap_ID.isin(blacklist)]
    print(len(filtered))
    filtered.to_csv('temp/filtered_fusions.csv', index=False)

    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # uploading to taiga
    tc.update_dataset(dataset_permaname=taiga_fusion[val],
                     upload_file_path_dict={
                         'temp/fusions.csv': 'TableCSV',
                         'temp/filtered_fusions.csv': 'TableCSV',
                         'temp/README', 'Raw'},
                      dataset_description=
    """
    # Internal Fusions

    for more information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str("rename")+"""

    BLACKLIST:
    """+str(blacklist))

    # virtual datasets
    AddToVirtual(virtual[val], taiga_fusion[val], files=[('CCLE_unfiltered_fusions', 'fusions'),('CCLE_fusions', 'filtered_fusions')])

not present
{'ACH-002196', 'ACH-002205', 'ACH-002289', 'ACH-001064', 'ACH-001641', 'ACH-002127', 'ACH-002344', 'ACH-002384', 'ACH-002383', 'ACH-002157', 'ACH-001037', 'ACH-001143', 'ACH-002216', 'ACH-001680', 'ACH-000727', 'ACH-002351', 'ACH-002207', 'ACH-002114', 'ACH-002300', 'ACH-000795', 'ACH-002377', 'ACH-002204', 'ACH-002360', 'ACH-002292', 'ACH-002058', 'ACH-002104', 'ACH-002314', 'ACH-002183', 'ACH-002213', 'ACH-002168', 'ACH-002382', 'ACH-002260', 'ACH-001208', 'ACH-002095', 'ACH-002200', 'ACH-002269', 'ACH-002387', 'ACH-002375', 'ACH-002129', 'ACH-001199', 'ACH-002312', 'ACH-000561', 'ACH-002282', 'ACH-002395', 'ACH-001338', 'ACH-002232', 'ACH-002267', 'ACH-002190', 'ACH-001502', 'ACH-001093', 'ACH-002336', 'ACH-002281', 'ACH-001137', 'ACH-002199', 'ACH-002208', 'ACH-002185', 'ACH-002276', 'ACH-002374', 'ACH-002396', 'ACH-001383', 'ACH-002237', 'ACH-001151', 'ACH-002119', 'ACH-002136', 'ACH-002359', 'ACH-001182', 'ACH-002399', 'ACH-002106', 'ACH-001187', 'ACH-002394', 'ACH-00

In [52]:
AddToVirtual('depmap-a0ab', "gene-fusions-8b7a", files=[('CCLE_fusions', 'fusions'),('CCLE_filtered_fusions', 'filtered_fusions')])

[('CCLE_fusions_unfiltered', 'gene-fusions-8b7a.16/fusions'), ('CCLE_fusions', 'gene-fusions-8b7a.16/filtered_fusions')]
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/api/datafile/e9e5878dd5db4d1585d0184db1854b46
hitting https://cds.team/taiga/a